<a href="https://colab.research.google.com/github/ilham-mukti/harga-rumah-jakarta/blob/main/Jakarta_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
!pip install folium

In [134]:
import geopandas as gpd
import folium 
from folium.plugins import MarkerCluster
from shapely import wkt

df = gpd.read_file("/content/Batas_Kec_DKI_Jakarta_Update.shp")
df.head()

,WADMKC,WADMKK,KDCPUM,KDPKAB,LUASWH,SRS_ID,geometry
0,MATRAMAN,JAKARTA TIMUR,317501,3175,485.76,None,"POLYGON ((106.87242 -6.19196, 106.87251 -6.191..."
1,PULO GADUNG,JAKARTA TIMUR,317502,3175,1490.49,None,"POLYGON ((106.89431 -6.17612, 106.89432 -6.176..."
2,JATINEGARA,JAKARTA TIMUR,317503,3175,1034.56,None,"POLYGON ((106.86757 -6.21497, 106.86785 -6.214..."
3,KRAMAT JATI,JAKARTA TIMUR,317504,3175,1307.12,None,"POLYGON ((106.87247 -6.25397, 106.87244 -6.254..."
4,PASAR REBO,JAKARTA TIMUR,317505,3175,1247.23,None,"POLYGON ((106.85795 -6.29221, 106.85812 -6.292..."


In [138]:
def convert_to_latlong(x):
  row = str(x)
  result = wkt.loads(row)
  lng = result.centroid.coords[0][0]
  lat = result.centroid.coords[0][1]
  return lng

In [137]:
df['latitude'] = df.geometry.apply(convert_to_latlong)

In [139]:
df['longitude'] = df.geometry.apply(convert_to_latlong)

In [140]:
df.head()

,WADMKC,WADMKK,KDCPUM,KDPKAB,LUASWH,SRS_ID,geometry,latitude,longitude
0,MATRAMAN,JAKARTA TIMUR,317501,3175,485.76,None,"POLYGON ((106.87242 -6.19196, 106.87251 -6.191...",-6.203613,106.864537
1,PULO GADUNG,JAKARTA TIMUR,317502,3175,1490.49,None,"POLYGON ((106.89431 -6.17612, 106.89432 -6.176...",-6.193199,106.889005
2,JATINEGARA,JAKARTA TIMUR,317503,3175,1034.56,None,"POLYGON ((106.86757 -6.21497, 106.86785 -6.214...",-6.227304,106.877425
3,KRAMAT JATI,JAKARTA TIMUR,317504,3175,1307.12,None,"POLYGON ((106.87247 -6.25397, 106.87244 -6.254...",-6.276307,106.865442
4,PASAR REBO,JAKARTA TIMUR,317505,3175,1247.23,None,"POLYGON ((106.85795 -6.29221, 106.85812 -6.292...",-6.326415,106.857256


In [ ]:
m = folium.Map(location=[-6.200000, 106.816666], zoom_start=11, tiles='openstreetmap')
m

In [ ]:
for _, r in df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.0001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'green'})
    folium.Popup(r['WADMKC']).add_to(geo_j)
    geo_j.add_to(m)
m

In [ ]:
m = folium.Map(location=[-6.232970,106.834572], tiles='OpenStreetMap', zoom_start=11.4)
#cluster = MarkerCluster().add_to(m)
m

In [142]:
for x in range(1, len(df.WADMKC)):
  folium.Marker(location=[df.latitude[x], df.longitude[x]], popup=df.WADMKC[x], tooltip=df.WADMKC[x], icon=folium.Icon(color='blue',icon="car", prefix='fa')).add_to(m)
m

In [143]:
df.to_csv("jakarta_kecamatan_maps.csv", index=False)